# Kapitel 4: Abstraktion, Operationen und Container-Verhalten

**Heutige Lernziele:**

Heute bringen wir unsere selbstgebauten Klassen auf die nächste Stufe. Wir sorgen dafür, dass sie sich wie vollwertige, intuitive Python-Objekte verhalten. Das bedeutet, wir können sie vergleichen (`==`, `<`), mit ihnen rechnen (`+`), sie in `for`-Schleifen verwenden und sogar "Bauvorschriften" für ihre Kindklassen festlegen. Dies ist der Tag, an dem unsere Klassen lernen, "Python zu sprechen".

* **Logische & Arithmetische Operationen:** Wir implementieren die Dunder-Methoden, die hinter den Operatoren `==`, `<`, `+` etc. stecken.
* **Abstrakte Basisklassen (ABCs):** Wir lernen, wie wir mit `@abstractmethod` eine Art "Vertrag" oder eine zwingend zu implementierende Schnittstelle für unsere Klassenhierarchien definieren.
* **Erweitern von Built-ins:** Wir schauen uns den einfachen Weg an, von `list` oder `dict` zu erben, um deren Verhalten anzupassen.
* **Das Container-Protokoll & Iteratoren:** Wir bauen die Magie von `len()`, `obj[key]` und `for item in obj` für unsere eigenen Klassen von Grund auf nach.

---
---

## **4.1: Objekte zum Leben erwecken: Operationen definieren**

In diesem ersten Block von Kapitel 4 geben wir unseren selbstgebauten Klassen fundamentale Fähigkeiten. Wir bringen ihnen bei, wie sie sich untereinander vergleichen und wie sie auf mathematische Operatoren wie `+` und `*` reagieren sollen. Wir sorgen dafür, dass sich unsere Objekte wie vollwertige, intuitive Datentypen in die Python-Sprache integrieren.

---

### **4.1.1 Vergleichsmethoden**

**Das Problem der Identität vs. Gleichheit**

Standardmäßig weiß Python nicht, wie es zwei Instanzen Ihrer Klasse inhaltlich vergleichen soll. Wenn Sie den `==`-Operator verwenden, passiert im Hintergrund dasselbe wie bei einer Prüfung mit dem `is`-Operator: Python vergleicht die **Speicheradressen** der Objekte.


In [1]:
class Student:
    def __init__(self, name):
        self.name = name

s1 = Student("Anna")
s2 = Student("Anna")
s3 = s1

# False! Obwohl der Inhalt (der Name) gleich ist, sind es zwei verschiedene Objekte an unterschiedlichen Orten im Speicher.
print(f"s1 == s2: {s1 == s2}") 

# True! s3 ist nur ein anderer Name für dasselbe Objekt im Speicher.
print(f"s1 == s3: {s1 == s3}") 

s1 == s2: False
s1 == s3: True



Dieses Verhalten ist selten das, was wir wollen. Wir wollen Python beibringen, unsere Objekte anhand ihrer **Attribute** zu vergleichen. Dafür gibt es die sogenannten **"Rich Comparison Methods"** (dt. "umfassende Vergleichsmethoden").

* `__eq__(self, other)`: Definiert das Verhalten für den Gleichheits-Operator `==`. Muss `True` oder `False` zurückgeben.
* `__ne__(self, other)`: Definiert das Verhalten für den Ungleichheits-Operator `!=`. Gibt oft einfach die Negation von `__eq__` zurück.
* `__lt__(self, other)`: Definiert "kleiner als" (`<`).
* `__gt__(self, other)`: Definiert "größer als" (`>`).
* `__le__(self, other)`: Definiert "kleiner-gleich" (`<=`).
* `__ge__(self, other)`: Definiert "größer-gleich" (`>=`).

Ein guter Stil ist es, immer zuerst mit `isinstance()` zu prüfen, ob das `other`-Objekt vom richtigen Typ ist. Wenn nicht, gibt man `NotImplemented` zurück. Das ist ein spezieller Wert, der Python signalisiert: "Ich weiß nicht, wie man diesen Vergleich durchführt; vielleicht weiß es der andere Typ."

**Profi-Tipp:** Wenn man `__eq__` und mindestens eine der Ordnungs-Methoden (z.B. `__lt__`) implementiert, kann man den `@functools.total_ordering`-Dekorator verwenden. Python generiert dann automatisch die restlichen Vergleichsmethoden (`<=`, `>`, `>=`), was Code spart.

---

### **Beispiel A : Vergleichbare Kreditkarten**

Zwei Kreditkartenobjekte sollen als gleich gelten, wenn ihre Kartennummer identisch ist, unabhängig vom Inhaber.


In [2]:


class Kreditkarte:
    def __init__(self, inhaber, kartennummer):
        self.inhaber = inhaber
        self.kartennummer = kartennummer
        
    def __eq__(self, other):
        # Schritt 1: Prüfen, ob das andere Objekt überhaupt eine Kreditkarte ist.
        if not isinstance(other, Kreditkarte):
            return NotImplemented
            
        # Schritt 2: Die eigentliche Vergleichslogik.
        # Wir vergleichen nur die Kartennummern.
        print(f"-> Vergleiche Kartennummer '{self.kartennummer}' mit '{other.kartennummer}'")
        return self.kartennummer == other.kartennummer

karte1 = Kreditkarte("Anna", "1234-5678-9012-3456")
karte2 = Kreditkarte("Ben", "9876-5432-1098-7654")
karte3 = Kreditkarte("Carla", "1234-5678-9012-3456") # Selbe Nummer wie Karte 1

# Jetzt wird unsere __eq__-Methode aufgerufen
print(f"Ist Karte 1 gleich Karte 2? {karte1 == karte2}")
print(f"Ist Karte 1 gleich Karte 3? {karte1 == karte3}")



-> Vergleiche Kartennummer '1234-5678-9012-3456' mit '9876-5432-1098-7654'
Ist Karte 1 gleich Karte 2? False
-> Vergleiche Kartennummer '1234-5678-9012-3456' mit '1234-5678-9012-3456'
Ist Karte 1 gleich Karte 3? True



---

### **Beispiel B : Sortierbare Software-Versionen**

Wir wollen Software-Versionen (z.B. "1.10.2") korrekt miteinander vergleichen können. Ein einfacher String-Vergleich würde hier versagen, da "1.10.0" alphabetisch vor "1.2.0" käme.


In [3]:


from functools import total_ordering

# Dieser Dekorator erspart uns die Implementierung von __le__, __gt__, __ge__
@total_ordering
class SoftwareVersion:
    def __init__(self, version_string):
        # Der Trick: Wir wandeln den String in ein Tupel von Integern um.
        # z.B. "1.10.2" -> (1, 10, 2)
        # Python kann Tupel von Zahlen perfekt vergleichen.
        self.version = tuple(map(int, version_string.split('.')))

    def __repr__(self):
        # Eine saubere Darstellung für die Ausgabe
        return f"Version({'.'.join(map(str, self.version))})"
        
    def __eq__(self, other):
        if not isinstance(other, SoftwareVersion):
            return NotImplemented
        return self.version == other.version

    def __lt__(self, other):
        if not isinstance(other, SoftwareVersion):
            return NotImplemented
        return self.version < other.version

# Instanzen erstellen
v1_2 = SoftwareVersion("1.2.0")
v1_10 = SoftwareVersion("1.10.0")
v2_0 = SoftwareVersion("2.0.0")
v1_10_clone = SoftwareVersion("1.10.0")

# Liste mit den Versionen
versionen = [v1_10, v2_0, v1_2, v1_10_clone]
print(f"Unsortierte Liste: {versionen}")

# Da wir __lt__ implementiert haben, weiß die sort()-Methode, wie sie vorgehen soll.
versionen.sort()
print(f"Sortierte Liste:   {versionen}")

# Dank @total_ordering funktionieren auch alle anderen Vergleiche automatisch
print(f"\nIst Version 1.10.0 > 1.2.0? {v1_10 > v1_2}") # True
print(f"Ist Version 1.10.0 >= 2.0.0? {v1_10 >= v2_0}") # False
print(f"Ist Version 1.10.0 == Klon? {v1_10 == v1_10_clone}") # True



Unsortierte Liste: [Version(1.10.0), Version(2.0.0), Version(1.2.0), Version(1.10.0)]
Sortierte Liste:   [Version(1.2.0), Version(1.10.0), Version(1.10.0), Version(2.0.0)]

Ist Version 1.10.0 > 1.2.0? True
Ist Version 1.10.0 >= 2.0.0? False
Ist Version 1.10.0 == Klon? True



---

### **4.1.2 Theorie & Beispiele: Arithmetische Methoden**

Genauso wie beim Vergleich weiß Python nicht, was es bedeutet, zwei unserer Objekte zu "addieren" oder mit einer Zahl zu "multiplizieren". Mit den arithmetischen Dunder-Methoden können wir das Verhalten von `+`, `*` etc. für unsere eigenen Datentypen definieren. Die wichtigste Regel ist dabei, dass diese Methoden in der Regel eine **neue Instanz** als Ergebnis zurückgeben sollten, anstatt die bestehenden zu verändern.

---

### **Beispiel A : `Playlist`-Klasse addieren**


In [4]:


class Playlist:
    def __init__(self, name, songs):
        self.name = name
        self.songs = songs 

    def __repr__(self):
        return f"Playlist('{self.name}', {self.songs})"

    def __add__(self, other):
        # Wir definieren, was '+' zwischen zwei Playlists bedeutet.
        if not isinstance(other, Playlist):
            return NotImplemented
        
        # Die Logik: Erstelle eine neue Playlist mit den kombinierten Song-Listen.
        kombinierte_songs = self.songs + other.songs
        neuer_name = f"{self.name} & {other.name}"
        return Playlist(neuer_name, kombinierte_songs)

rock_playlist = Playlist("Rock Classics", ["Stairway to Heaven", "Bohemian Rhapsody"])
pop_playlist = Playlist("Pop Hits", ["Blinding Lights"])

# Der '+' Operator ruft im Hintergrund rock_playlist.__add__(pop_playlist) auf
gemischte_playlist = rock_playlist + pop_playlist

print(gemischte_playlist)



Playlist('Rock Classics & Pop Hits', ['Stairway to Heaven', 'Bohemian Rhapsody', 'Blinding Lights'])



---

### **Beispiel B : `RGBFarbe`-Klasse mischen und aufhellen**


In [5]:


class RGBFarbe:
    def __init__(self, r, g, b):
        # Werte werden auf den Bereich 0-255 beschränkt
        self.r = min(255, max(0, r))
        self.g = min(255, max(0, g))
        self.b = min(255, max(0, b))

    def __repr__(self):
        return f"RGB({self.r}, {self.g}, {self.b})"
        
    # Definiert das Verhalten für 'farbe1 + farbe2'
    def __add__(self, other):
        if not isinstance(other, RGBFarbe):
            return NotImplemented
        # Farben mischen durch Mittelwertbildung der Komponenten
        neues_r = int((self.r + other.r) / 2)
        neues_g = int((self.g + other.g) / 2)
        neues_b = int((self.b + other.b) / 2)
        return RGBFarbe(neues_r, neues_g, neues_b)

    # Definiert das Verhalten für 'farbe * zahl'
    def __mul__(self, skalar):
        if not isinstance(skalar, (int, float)):
            return NotImplemented
        # Farbe aufhellen/abdunkeln
        neues_r = int(self.r * skalar)
        neues_g = int(self.g * skalar)
        neues_b = int(self.b * skalar)
        return RGBFarbe(neues_r, neues_g, neues_b)

# Farben definieren
rot = RGBFarbe(255, 0, 0)
blau = RGBFarbe(0, 0, 255)
grau = RGBFarbe(128, 128, 128)

# Farben mischen mit '+'
lila = rot + blau
print(f"{rot} + {blau} = {lila}")

# Farbe aufhellen mit '*'
hellgrau = grau * 1.5
print(f"{grau} * 1.5 = {hellgrau}")



RGB(255, 0, 0) + RGB(0, 0, 255) = RGB(127, 0, 127)
RGB(128, 128, 128) * 1.5 = RGB(192, 192, 192)



---

### **4.1.3 Transfer auf das `Raumschiff`-Projekt**

Jetzt wenden wir beide Konzepte auf unsere Projekte an. Wir machen `Raumschiffe` vergleichbar und `Flotten` addierbar.


In [6]:


from functools import total_ordering

@total_ordering
class Schiffsklasse:
    def __init__(self, name, huelle_staerke):
        self.name = name
        self.huelle_staerke = huelle_staerke
    
    def __repr__(self):
        return f"{self.name}({self.huelle_staerke}%)"
    
    # Notwendig für die Sortierung und @total_ordering
    def __eq__(self, other):
        if not isinstance(other, Schiffsklasse): return NotImplemented
        return self.huelle_staerke == other.huelle_staerke
        
    def __lt__(self, other):
        if not isinstance(other, Schiffsklasse): return NotImplemented
        return self.huelle_staerke < other.huelle_staerke

class Flotte:
    def __init__(self, name, schiffe=None):
        self.name = name
        # Standardwert ist eine leere Liste, falls keine übergeben wird
        self.schiffe = schiffe if schiffe is not None else []
    
    def __repr__(self):
        return f"Flotte '{self.name}': {self.schiffe}"
        
    def __add__(self, other):
        if not isinstance(other, Flotte): return NotImplemented
        # Kombiniert die Schiffslisten beider Flotten in eine neue Flotte
        neue_schiffsliste = self.schiffe + other.schiffe
        neuer_name = f"Vereinigte Flotte ({self.name} & {other.name})"
        return Flotte(neuer_name, neue_schiffsliste)

# 1. Schiffe vergleichen
enterprise = Schiffsklasse("Enterprise", 100)
defiant = Schiffsklasse("Defiant", 150)
voyager = Schiffsklasse("Voyager", 100)

print(f"Ist Defiant stärker als Enterprise? {defiant > enterprise}")
print(f"Ist Enterprise gleich stark wie Voyager? {enterprise == voyager}")

# 2. Flotten fusionieren
flotte1 = Flotte("Flotte 1", [enterprise, voyager])
flotte2 = Flotte("Flotte 2", [defiant])

angriffs_flotte = flotte1 + flotte2
print(angriffs_flotte)



Ist Defiant stärker als Enterprise? True
Ist Enterprise gleich stark wie Voyager? True
Flotte 'Vereinigte Flotte (Flotte 1 & Flotte 2)': [Enterprise(100%), Voyager(100%), Defiant(150%)]



---
---

## **4.2: Klassen mit Regeln: Abstraktion und Erweiterung**

In diesem Kapitel geben wir unseren Klassendesigns eine professionelle Struktur. Wir lernen, wie wir formale "Verträge" für unsere Klassenhierarchien erstellen (Abstraktion) und wie wir die mächtigen, eingebauten Datentypen von Python als Fundament für unsere eigenen, spezialisierten Klassen nutzen können (Erweiterung).

---
---

### **4.2.1 Abstrakte Basisklassen (ABCs)**

**Warum Duck Typing in großen Projekten gefährlich sein kann**

Wir haben in Kapitel 2 über "Duck Typing" gesprochen: "Wenn es geht wie eine Ente und quakt wie eine Ente, dann muss es eine Ente sein." Dieses Prinzip ist flexibel und sehr "pythonisch", hat aber in großen, komplexen Systemen oder bei der Entwicklung von Frameworks entscheidende Nachteile:

1.  **Fehler werden erst zur Laufzeit entdeckt:** Stellen Sie sich eine Funktion vor, die von einem Objekt erwartet, dass es eine `.exportieren()`-Methode hat. Wenn ein Entwickler nun eine Klasse schreibt und die Methode versehentlich `exportiren()` nennt (ein Tippfehler), wird das Programm erst abstürzen, wenn die Funktion tatsächlich aufgerufen wird. Es gibt keine vorherige Warnung.
2.  **Keine klare Schnittstellendefinition:** Woher weiß ein anderer Entwickler, der Ihre Klasse verwenden möchte, welche Methoden er *unbedingt* implementieren muss, damit sein Code mit Ihrem System kompatibel ist? Er muss Ihren Code oder Ihre Dokumentation lesen und hoffen, dass er nichts übersieht. Es gibt keinen formellen "Vertrag".

**Die Lösung: Abstrakte Basisklassen (ABCs)**

ABCs sind die formale Lösung für dieses Problem. Sie ermöglichen es uns, eine "Schablone" oder einen **Vertrag** für unsere Kindklassen zu definieren.

* **Konzept:** Eine ABC ist eine spezielle Klasse, die nicht dazu gedacht ist, selbst instanziiert zu werden. Sie dient ausschließlich als Vorlage.
* **Mechanismus:** Innerhalb einer ABC verwenden wir den Dekorator `@abstractmethod` aus dem `abc`-Modul, um Methoden zu deklarieren, die jede konkrete Kindklasse **implementieren muss**.
* **Der Vertrag:** Wenn eine Klasse von einer ABC erbt, geht sie einen Vertrag ein. Sie verpflichtet sich, alle mit `@abstractmethod` markierten Methoden selbst zu implementieren.
* **Die Konsequenz:** Versucht man, eine Instanz einer Kindklasse zu erstellen, die diesen Vertrag nicht vollständig erfüllt (also nicht alle abstrakten Methoden implementiert hat), löst Python sofort einen `TypeError` aus. Der Fehler wird also frühzeitig bei der Erstellung des Objekts gefunden, nicht erst spät zur Laufzeit.

---

### **Beispiel A : Die `Grafikobjekt`-Schnittstelle**

Wir wollen sicherstellen, dass jedes geometrische Objekt in unserem Programm gezeichnet werden kann.


In [7]:


from abc import ABC, abstractmethod

# Dies ist eine Abstrakte Basisklasse, die den Vertrag definiert.
class Grafikobjekt(ABC): 
    def __init__(self, farbe):
        self.farbe = farbe
        
    @abstractmethod
    def zeichnen(self):
        """Jedes Grafikobjekt MUSS diese Methode haben."""
        pass # Eine abstrakte Methode hat keinen eigenen Code.

    def info_anzeigen(self):
        # Eine ABC kann auch normale, konkrete Methoden haben!
        print(f"Dies ist ein Grafikobjekt mit der Farbe {self.farbe}.")

# Versuch, eine Instanz der ABC zu erstellen -> Führt zu einem Fehler!
# unvollstaendig = Grafikobjekt("Rot") # TypeError: Can't instantiate abstract class Grafikobjekt

# --- Konkrete Implementierungen ---

class Kreis(Grafikobjekt):
    def zeichnen(self): # Erfüllt den Vertrag
        print(f"Zeichne einen {self.farbe}en Kreis: O")

class Quadrat(Grafikobjekt):
    def zeichnen(self): # Erfüllt den Vertrag
        print(f"Zeichne ein {self.farbe}es Quadrat: []")

class Linie(Grafikobjekt):
    # Diese Klasse implementiert 'zeichnen()' absichtlich NICHT!
    pass

# Testen der gültigen Klassen
k = Kreis("Blau")
k.info_anzeigen()
k.zeichnen()

q = Quadrat("Grün")
q.zeichnen()

# Versuch, die ungültige Klasse zu instanziieren -> Führt zu einem Fehler!
print("\nVersuch, eine Linie zu erstellen:")
try:
    l = Linie("Schwarz")
except TypeError as e:
    print(f"FEHLER wie erwartet: {e}")



Dies ist ein Grafikobjekt mit der Farbe Blau.
Zeichne einen Blauen Kreis: O
Zeichne ein Grünes Quadrat: []

Versuch, eine Linie zu erstellen:
FEHLER wie erwartet: Can't instantiate abstract class Linie without an implementation for abstract method 'zeichnen'



---

### **Beispiel B : Ein `Plugin`-System**

Wir definieren eine robuste Schnittstelle für ein Plugin-System. Jedes gültige Plugin muss sich initialisieren, eine Hauptfunktion ausführen und sich sauber beenden können.


In [8]:


from abc import ABC, abstractmethod

class PluginBase(ABC):
    @abstractmethod
    def initialisieren(self, config: dict):
        """Bereitet das Plugin für die Ausführung vor."""
        pass
    
    @abstractmethod
    def ausfuehren(self):
        """Führt die Hauptlogik des Plugins aus."""
        pass
        
    @abstractmethod
    def beenden(self):
        """Räumt nach der Ausführung auf."""
        pass

# --- Konkrete Implementierungen ---

class DatenAnalysePlugin(PluginBase):
    def initialisieren(self, config: dict):
        self.datenquelle = config.get("quelle")
        print(f"AnalysePlugin: Initialisiert mit Datenquelle '{self.datenquelle}'.")

    def ausfuehren(self):
        print("AnalysePlugin: Führe komplexe Datenanalyse durch...")

    def beenden(self):
        print("AnalysePlugin: Analyse abgeschlossen, Ressourcen freigegeben.")

class WetterWidgetPlugin(PluginBase):
    def initialisieren(self, config: dict):
        self.stadt = config.get("stadt", "Berlin")
        print(f"WetterWidget: Initialisiert für Stadt '{self.stadt}'.")

    def ausfuehren(self):
        print(f"WetterWidget: Zeige aktuelles Wetter für {self.stadt} an.")
        
    # Diese Klasse VERGISST, die beenden()-Methode zu implementieren!
    # def beenden(self):
    #     pass

# Testen
plugin1 = DatenAnalysePlugin()
plugin1.initialisieren({"quelle": "sensordaten.csv"})
plugin1.ausfuehren()
plugin1.beenden()

print("\nVersuch, das Wetter-Plugin zu instanziieren:")
try:
    plugin2 = WetterWidgetPlugin()
except TypeError as e:
    print(f"FEHLER wie erwartet: {e}")




AnalysePlugin: Initialisiert mit Datenquelle 'sensordaten.csv'.
AnalysePlugin: Führe komplexe Datenanalyse durch...
AnalysePlugin: Analyse abgeschlossen, Ressourcen freigegeben.

Versuch, das Wetter-Plugin zu instanziieren:
FEHLER wie erwartet: Can't instantiate abstract class WetterWidgetPlugin without an implementation for abstract method 'beenden'



---
---

### **4.2.2 Von Built-ins erben**

### **Warum und wann sollte man von `list` oder `dict` erben?**

Python bietet uns eine Abkürzung: Wenn unser eigenes Objekt im Kern eigentlich nur eine leicht veränderte Liste oder ein leicht verändertes Dictionary ist, können wir direkt von diesen Klassen erben.

**Warum sollte man das tun?**
* **Effizienz und Bequemlichkeit:** Wir bekommen die gesamte, in C implementierte und hochoptimierte Funktionalität des Built-in-Typs "geschenkt" (Sortieren, Slicing, Iteration, etc.). Wir müssen nur die wenigen Methoden überschreiben, die wir anpassen wollen.
* **Bekannte Schnittstelle:** Unser Objekt verhält sich für andere Entwickler sofort vertraut, da es alle bekannten Listen- oder Dictionary-Methoden unterstützt.

**Vererbung ("is a") vs. Komposition ("has a")**
* **Vererbung:** Verwenden Sie diesen Weg, wenn Ihr Objekt wirklich eine **spezialisierte Art** von Liste oder Dictionary *ist*. Eine `UniqueList` *ist eine* Liste, aber mit einer zusätzlichen Regel.
* **Komposition:** Verwenden Sie diesen Weg, wenn Ihr Objekt eine Liste oder ein Dictionary intern nur als **Hilfsmittel** *hat*, aber nach außen eine ganz andere Schnittstelle bietet. Unsere `Flotte`-Klasse *hat eine* Liste von Schiffen, aber sie *ist* keine Liste (sie hat z.B. einen Namen).

---

### **Beispiel A: Die `UniqueList`**

Wir erstellen eine Liste, die keine doppelten Einträge erlaubt, indem wir von `list` erben und nur die Methoden überschreiben, die Elemente hinzufügen.


In [10]:



class UniqueList(list):
    def append(self, item):
        # Eigene Regel: Nur hinzufügen, wenn noch nicht vorhanden.
        if item not in self:
            # super() ruft die ursprüngliche append-Methode der Elternklasse (list) auf.
            super().append(item)
    
    def extend(self, iterable):
        # Wir überschreiben auch extend, um Duplikate zu filtern
        for item in iterable:
            self.append(item) # Ruft unsere eigene, sichere append-Methode auf

# Testen
ul = UniqueList()
ul.append(10)
ul.append(20)
ul.append(10) # Wird ignoriert
print(f"Nach append: {ul}")

ul.extend([30, 20, 40]) # 20 wird ignoriert
print(f"Nach extend: {ul}")

# Alle anderen Listen-Methoden funktionieren weiterhin perfekt!
print(f"Erstes Element: {ul[0]}")
print(f"Länge: {len(ul)}")



Nach append: [10, 20]
Nach extend: [10, 20, 30, 40]
Erstes Element: 10
Länge: 4



---

### **Beispiel B: Das `CaseInsensitiveDict`**

Wir erstellen ein Dictionary, das bei Schlüsselnamen nicht zwischen Groß- und Kleinschreibung unterscheidet.



In [11]:

class CaseInsensitiveDict(dict):
    # __setitem__ wird bei Zuweisungen wie d['key'] = value aufgerufen
    def __setitem__(self, key, value):
        # Wir wandeln den Schlüssel immer in Kleinbuchstaben um, bevor wir ihn speichern.
        super().__setitem__(key.lower(), value)

    # __getitem__ wird bei Lesezugriffen wie d['key'] aufgerufen
    def __getitem__(self, key):
        # Wir suchen ebenfalls nach der kleingeschriebenen Version des Schlüssels.
        return super().__getitem__(key.lower())

# Testen
cid = CaseInsensitiveDict()
cid['Name'] = "Anna"
cid['Stadt'] = "Berlin"

print(f"Zugriff mit 'Name': {cid['Name']}")
print(f"Zugriff mit 'name': {cid['name']}") # Funktioniert!
print(f"Zugriff mit 'STADT': {cid['STADT']}") # Funktioniert auch!

print(f"Das gesamte Dictionary: {cid}")



Zugriff mit 'Name': Anna
Zugriff mit 'name': Anna
Zugriff mit 'STADT': Berlin
Das gesamte Dictionary: {'name': 'Anna', 'stadt': 'Berlin'}



---
---

### **4.2.3 Transfer auf das `Raumschiff`-Projekt**

**Anwendung von ABCs:**
Jedes `Antriebssystem` an Bord eines Raumschiffs muss eine `aktivieren`-Methode haben. Wir erzwingen dies mit einer ABC.


In [12]:


from abc import ABC, abstractmethod

class Antriebssystem(ABC):
    @abstractmethod
    def aktivieren(self, energie):
        """Aktiviert den Antrieb mit einer bestimmten Energiemenge."""
        pass

class WarpAntrieb(Antriebssystem):
    def aktivieren(self, energie):
        if energie > 50:
            print(f"Warp-Antrieb aktiviert mit {energie} Einheiten. Sprung wird vorbereitet!")
        else:
            print("Nicht genug Energie für den Warp-Sprung.")

class Impulsantrieb(Antriebssystem):
    def aktivieren(self, energie):
        print(f"Impulsantrieb mit {energie} Einheiten aktiviert.")

# Das Raumschiff kann nun sicher sein, dass jedes Antriebsobjekt .aktivieren() kann.
enterprise_antrieb = WarpAntrieb()
enterprise_antrieb.aktivieren(75)



Warp-Antrieb aktiviert mit 75 Einheiten. Sprung wird vorbereitet!



**Anwendung von Subclassing Built-ins:**
Wir erstellen ein `SchiffsRegister`, das sich wie ein Dictionary verhält, aber nur `Raumschiff`-Objekte als Werte akzeptiert.


In [13]:


# Wir benötigen eine Basis-Raumschiff-Klasse für isinstance()
class Raumschiff:
    def __init__(self, name):
        self.name = name

class SchiffsRegister(dict):
    def __setitem__(self, key, value):
        # Eigene Regel: Nur Raumschiffe erlauben
        if not isinstance(value, Raumschiff):
            raise TypeError("Nur Raumschiff-Objekte können dem Register hinzugefügt werden.")
        super().__setitem__(key, value)

# Testen
register = SchiffsRegister()
enterprise = Raumschiff("Enterprise")
voyager = Raumschiff("Voyager")

register["NCC-1701-D"] = enterprise # Funktioniert
register["NCC-74656"] = voyager # Funktioniert

try:
    register["SHUTTLE-01"] = "Ein Shuttle" # Löst einen Fehler aus
except TypeError as e:
    print(f"FEHLER wie erwartet: {e}")
    
print(f"\nAktuelles Register: {register}")



FEHLER wie erwartet: Nur Raumschiff-Objekte können dem Register hinzugefügt werden.

Aktuelles Register: {'NCC-1701-D': <__main__.Raumschiff object at 0x112349400>, 'NCC-74656': <__main__.Raumschiff object at 0x112388690>}



---
---

## **4.3 Klassen wie Container: Das Protokoll für `len()`, `[]` und `for`**

Bisher sind unsere Objekte noch recht "dumm". Sie wissen nicht, wie sie auf grundlegende Python-Operationen reagieren sollen, die wir von Listen oder Dictionaries gewohnt sind. In diesem Abschnitt bringen wir unseren Klassen bei, sich wie vollwertige, native Python-Container zu verhalten.

---

### **Das Container-Protokoll (Teil 1: Größe und Indizierung)**

Wie können wir `len(mein_objekt)` oder `mein_objekt[0]` für unsere eigenen Klassen ermöglichen? Indem wir die entsprechenden Dunder-Methoden des "Container-Protokolls" implementieren.

* **`__len__(self)`**
    * **Zweck:** Definiert das Verhalten für die globale Funktion `len()`.
    * **Anforderung:** Muss eine nicht-negative Ganzzahl (`int`) zurückgeben.
    * **Hintergrund:** Wenn Sie `len(obj)` aufrufen, ruft Python intern `obj.__len__()` auf.

* **`__getitem__(self, key)`**
    * **Zweck:** Definiert das Verhalten für den **Lesezugriff** mit eckigen Klammern: `obj[key]`.
    * **Anforderung:** Der Parameter `key` ist das, was in den Klammern steht (z.B. ein Index `0` oder ein Dictionary-Schlüssel `"name"`). Die Methode sollte den entsprechenden Wert zurückgeben. Wenn der Schlüssel ungültig ist, sollte sie einen `IndexError` (für Sequenzen) oder `KeyError` (für Mappings) auslösen.
    * **Hintergrund:** `x = obj[key]` wird zu `x = obj.__getitem__(key)`.

* **`__setitem__(self, key, value)`**
    * **Zweck:** Definiert das Verhalten für den **Schreibzugriff** mit eckigen Klammern: `obj[key] = value`.
    * **Hintergrund:** `obj[key] = value` wird zu `obj.__setitem__(key, value)`.

* **`__delitem__(self, key)`**
    * **Zweck:** Definiert das Verhalten für das Löschen mit `del`: `del obj[key]`.
    * **Hintergrund:** `del obj[key]` wird zu `obj.__delitem__(key)`.

---

### **Beispiel: Ein `Regal`-Klasse für Bücher**

Wir bauen eine `Regal`-Klasse, die eine Sammlung von Büchern verwaltet und sich wie eine Liste verhält.


In [ ]:


class Regal:
    def __init__(self, standort):
        self.standort = standort
        # Wir verwenden eine interne Liste zur Speicherung der Bücher
        self._buecher = []

    def buch_hinzufuegen(self, buch_titel):
        self._buecher.append(buch_titel)
        print(f"'{buch_titel}' wurde zu Regal '{self.standort}' hinzugefügt.")

    def __len__(self):
        print("-> __len__ wird aufgerufen...")
        return len(self._buecher)

    def __getitem__(self, index):
        print(f"-> __getitem__ wird mit Index {index} aufgerufen...")
        return self._buecher[index]

    def __setitem__(self, index, buch_titel):
        print(f"-> __setitem__ wird aufgerufen: Ersetze Buch an Position {index}...")
        self._buecher[index] = buch_titel
        
    def __repr__(self):
        return f"Regal('{self.standort}', {self._buecher})"

# Testen
buecherregal = Regal("Wohnzimmer")
buecherregal.buch_hinzufuegen("Der Herr der Ringe")
buecherregal.buch_hinzufuegen("Stolz und Vorurteil")

# Testen von __len__
print(f"\nAnzahl der Bücher im Regal: {len(buecherregal)}")

# Testen von __getitem__
print(f"Das erste Buch ist: '{buecherregal[0]}'")

# Testen von __setitem__
buecherregal[1] = "Moby Dick"
print(f"Das Regal nach der Änderung: {buecherregal}")




---
### **Das Iterator-Protokoll – Die `for`-Schleife entmystifiziert**

Obwohl unsere `Regal`-Klasse jetzt `len()` und `[]` unterstützt, würde eine `for`-Schleife (`for buch in buecherregal:`) noch nicht elegant funktionieren. Python würde zwar über `__getitem__` und einen Index von 0 bis `len()-1` iterieren können, aber das ist nur ein "Fallback"-Mechanismus. Der richtige, effiziente Weg ist das **Iterator-Protokoll**.

**Wie funktioniert eine `for`-Schleife im Hintergrund?**

1.  Wenn Python `for item in mein_objekt:` sieht, ruft es zuerst `iter(mein_objekt)` auf. Diese Funktion versucht, die `__iter__()`-Methode des Objekts aufzurufen. Ein Objekt, das `__iter__` hat, nennt man ein **iterierbares Objekt (Iterable)**.
2.  Die `__iter__()`-Methode muss ein **Iterator-Objekt** zurückgeben.
3.  Ein **Iterator** ist jedes Objekt, das eine `__next__()`-Methode hat.
4.  Die `for`-Schleife ruft dann wiederholt die `__next__()`-Methode dieses Iterator-Objekts auf, um das nächste Element zu erhalten.
5.  Wenn keine Elemente mehr da sind, muss die `__next__()`-Methode eine `StopIteration`-Exception auslösen. Dieser Fehler wird von der `for`-Schleife intern und unsichtbar abgefangen und beendet die Schleife sauber.

**Muster 1: Objekt ist sein eigener Iterator (einfach)**
Die Klasse implementiert sowohl `__iter__` als auch `__next__`. `__iter__` gibt einfach `self` zurück.

**Muster 2: Separates Iterator-Objekt (fortgeschritten & flexibler)**
Die Klasse implementiert nur `__iter__` und gibt eine Instanz einer *anderen* Klasse zurück, die die `__next__`-Logik enthält. Dies erlaubt mehrere, unabhängige Iterationen über dasselbe Objekt.

---

### **Beispiel A : Eine `Countdown`-Klasse als Iterator**

Dieses Beispiel zeigt Muster 1: Das Objekt ist sein eigener Iterator.


In [ ]:


class Countdown:
    def __init__(self, start):
        self.current = start
        print(f"Countdown von {start} initialisiert.")

    def __iter__(self):
        # Das Objekt gibt sich selbst als seinen eigenen Iterator zurück.
        print("-> __iter__ aufgerufen. Die Iteration beginnt.")
        return self

    def __next__(self):
        print("-> __next__ aufgerufen...")
        if self.current < 1:
            # Keine Zahlen mehr übrig, wir signalisieren das Ende.
            print("-> StopIteration wird ausgelöst.")
            raise StopIteration
        else:
            # Gibt den aktuellen Wert zurück und zählt für den nächsten Aufruf herunter.
            val = self.current
            self.current -= 1
            return val

print("Countdown von 3 starten:")
for zahl in Countdown(3):
    print(zahl)

# Nochmaliger Versuch würde nicht funktionieren, da der Zähler bei 0 ist.
# Man müsste eine neue Instanz erstellen.




---

### **Beispiel B : Ein `Satz`-Objekt mit separatem `WortIterator`**

Dieses Beispiel zeigt Muster 2. Es ist flexibler, weil wir mehrfach über denselben Satz iterieren können.


In [ ]:


# Der Iterator: Enthält die Logik für die Iteration
class WortIterator:
    def __init__(self, worte):
        self._worte = worte
        self._index = 0

    def __next__(self):
        if self._index < len(self._worte):
            wort = self._worte[self._index]
            self._index += 1
            return wort
        else:
            raise StopIteration

# Das iterierbare Objekt: Erzeugt bei Bedarf neue Iteratoren
class Satz:
    def __init__(self, text):
        self._worte = text.split()
    
    def __iter__(self):
        # Gibt bei jedem Aufruf einen NEUEN, frischen Iterator zurück
        print("-> __iter__ von Satz aufgerufen. Erzeuge einen neuen WortIterator.")
        return WortIterator(self._worte)

# Testen
mein_satz = Satz("Ein Fuchs springt ueber einen Hund")

print("Erste Iteration:")
for wort in mein_satz:
    print(wort, end=' ')
print("\n")

print("Zweite, unabhängige Iteration:")
for wort in mein_satz:
    print(wort.upper(), end=' ')
print()





---
---

### **4.3.3 Transfer auf das `Raumschiff`-Projekt: Eine voll funktionsfähige `Flotte`**

Wir rüsten unsere `Flotte`-Klasse, die wir in Abschnitt 4.1 erstellt haben, zu einem vollwertigen, iterierbaren Container auf. Wir verwenden hier das einfache Muster, bei dem das Objekt sein eigener Iterator ist.


In [ ]:


# Schiffsklasse als Dummy für das Beispiel
class Schiffsklasse:
    def __init__(self, name): self.name = name
    def __repr__(self): return self.name

class Flotte:
    def __init__(self, name):
        self.name = name
        self._schiffe = []
        self._index = 0

    def add_schiff(self, schiff):
        if isinstance(schiff, Schiffsklasse):
            self._schiffe.append(schiff)
        else:
            print("Fehler: Nur Schiffsklasse-Objekte können hinzugefügt werden.")
        
    # --- Container-Protokoll Teil 1 ---
    def __len__(self):
        return len(self._schiffe)

    def __getitem__(self, position):
        return self._schiffe[position]
    
    # --- Container-Protokoll Teil 2 (Iteration) ---
    def __iter__(self):
        # Setzt den Index für den Start einer neuen Iteration zurück
        self._index = 0
        return self
        
    def __next__(self):
        if self._index < len(self._schiffe):
            schiff = self._schiffe[self._index]
            self._index += 1
            return schiff
        else:
            raise StopIteration
            
    def __repr__(self):
        return f"Flotte '{self.name}' mit {len(self)} Schiffen"

# Testen
test_flotte = Flotte("5. Flotte")
test_flotte.add_schiff(Schiffsklasse("Voyager"))
test_flotte.add_schiff(Schiffsklasse("Discovery"))
test_flotte.add_schiff(Schiffsklasse("Titan"))

# Test __len__
print(f"Anzahl der Schiffe in der Flotte: {len(test_flotte)}")

# Test __getitem__
print(f"Das zweite Schiff ist: {test_flotte[1]}")

# Test der Iteration
print("\nAlle Schiffe der Flotte mit einer for-Schleife durchlaufen:")
for s in test_flotte:
    print(f"- Kommunikationskanal zu '{s.name}' geöffnet.")





---
---

## **4.4: Gesammelte Tages-Challenge für das Schüler-Projekt (`Smart Grid`)**

Jetzt kombinieren wir alle heutigen Konzepte in Ihrem Projekt.

**Ihre umfassende Aufgabe:**

1.  **Vergleichbare Energieerzeuger:**
    * Implementieren Sie die Vergleichsmethoden (`__eq__` und `__lt__`) für Ihre `Energieerzeuger`-Basisklasse. Der Vergleich soll anhand der `max_leistung` erfolgen. Nutzen Sie `@functools.total_ordering`, um sich Arbeit zu sparen.
    * Erstellen Sie eine Liste mit mehreren `Solaranlage`- und `Windkraftwerk`-Instanzen und sortieren Sie diese Liste, um zu zeigen, dass der Vergleich funktioniert.

2.  **Ein `Verteilerknoten` als Container:**
    * Erstellen Sie eine neue Klasse `Verteilerknoten`.
    * Ein `Verteilerknoten` soll eine Liste von angeschlossenen `Energieerzeuger`-Objekten verwalten.
    * Implementieren Sie das komplette Container-Protokoll:
        * `__len__`: Gibt die Anzahl der angeschlossenen Erzeuger zurück.
        * `__getitem__`: Ermöglicht den Zugriff auf einen Erzeuger per Index.
        * Das **Iterator-Protokoll** (`__iter__` und `__next__`), um über die angeschlossenen Erzeuger zu iterieren.

3.  **Eine Abstrakte `Wartungs`-Schnittstelle:**
    * Erstellen Sie eine ABC `Wartungsfaehig` mit einer `@abstractmethod` namens `wartungs_check_durchfuehren()`, die `True` (wenn alles okay ist) oder `False` (wenn Wartung nötig ist) zurückgibt.
    * Lassen Sie Ihre `Energieerzeuger`-Basisklasse von dieser ABC erben und implementieren Sie die Methode dort (sie kann einfach `True` zurückgeben).
    * Überschreiben Sie die Methode in den Kindklassen (`Solaranlage`, etc.) mit einer spezifischeren Logik (z.B. eine Solaranlage muss gewartet werden, wenn ihre `panel_flaeche` sehr groß ist).

4.  **Alles testen:**
    * Erstellen Sie einen `Verteilerknoten`, fügen Sie mehrere `Energieerzeuger` hinzu und testen Sie alle Container-Funktionen (`len`, Index-Zugriff, `for`-Schleife).
    * Schreiben Sie eine externe Funktion `starte_wartungsrunde(knoten)`. Diese Funktion soll über den `Verteilerknoten` iterieren und für jeden Erzeuger, bei dem der `wartungs_check_durchfuehren()` `False` zurückgibt, eine Nachricht ausgeben, z.B. "`{erzeuger.name}` muss gewartet werden!"`.
